# Manual Scheduling

using `nodeName` field in pod definition

by setting the `nodeName` field to the Name of the node


can use node name only when creating a Pod not after creating a pod.

k8s won't allow to change the `nodeName` property of an already existing pod

To change the `nodeName` of an already existing pod, create a `BInding` object and send a post request to the `binding` api of the Pod.

```
apiVersion: v1
kind: Binding
metadata:
  name: nginx
target:
  apiVersion: v1
  kind: Node
  name: node02
```

```
curl --header "Content-Type:application/json" --request POST --data '{"apiVersion": "v1", "kind": "Binding", ...}' \
http://<server>/api/v1/namespaces/default/pods/<pod-name>/binding/
```




---

# Labels and Selectors

## Labels

add labels in Pod metadata

```
...
metadata:
  labels:
    key1: val1
    key2: val2
...

```

selection

```
kubectl get pods --selector <key>=<value>

kubectl get pods --selector <key>=<value>,<key>=<value>,<key>=<value>
```

## Annotations

used to record information on objects

```
...
metadata:
  labels:
    app: App1
    function: front-end
  annotations:
    buildversion: 1.2.3
...

```
---

## Taints and Tolerations

taint: mark a node with a a requirement so that all normal pods are prevented from being scheduled on it
toleration: mark a pod with a toleration so that it can tolerate a taint on a node and be scheduled on that node

taints are set on nodes and tolerations are set on pods


**taint**
```
kubect taint nodes <node-name> <key>=<value>:<taint-effect>
```

taint-effect

1. `NoSchedule`
1. `PreferNoSchedule`
1. `NoExecute`

eg:

```
kubectl taint nodes node1 app=app1:NoSchedule

```

```
$ kc get node master  -o yaml | grep -A 3 taint
```
```
  taints:
  - effect: NoSchedule
    key: node-role.kubernetes.io/master

```

**toleration**

```
...
spec:
  ...
  tolerations:
  - key: "app"
    value: "app1"
    effect: "NoSchedule"
  ...
...
```
---

## Node Selector

in pod definition
```
spec:
  containers:
  ...

  nodeSelector:
    size: large # (labels assigned to Nodes)
```

```
kubectl label nodes <node-name> <key>=<value>
```

NodeSlectors
- cannot handle complex conditions (large or small, any but small)
---


## Node Affinity

pod specification
```
spec:
  containers:
  - 
  ...
  affinity:
    nodeAffinity:
      requiredDuringSchedulingIgnoredDuringExecution:
        nodeSelectorTerms:
        - matchExpressions:
          - key: size
            operator: In
            value:
            - Large
```

```
        - matchExpressions:
          - key: size
            operator: NotIn
            value:
            - Small
```

```
        - matchExpressions:
          - key: size
            operator: Exists
```

### NodeAffinityTypes

**available**
- requiredDuringSchedulingIgnoredDuringExecution
- preferredDuringSchedulingIgnoredDuringExecution

**planned**
- requiredDuringSchedulingrequiredDuringExecution
---

## Resource Requriements

by default  Pod is assumed to need 0.5 cpu and 256MI mem

resources and limits are set for each container

---

## Editing Pods

you CANNOT edit specifications of an existing POD other than the below.

- spec.containers[*].image

- spec.initContainers[*].image

- spec.activeDeadlineSeconds

- spec.tolerations

For example you cannot edit the environment variables, service accounts, resource limits

## Edit Deployments

With Deployments you can easily edit any field/property of the POD template. Since the pod template is a child of the deployment specification,  with every change the deployment will automatically delete and create a new pod with the new changes. So if you are asked to edit a property of a POD part of a deployment you may do that simply by running the command

`kubectl edit deployment my-deployment`

## DaemonSets

Daemonset uses the default scheduler and nodeaffinity rules to schedule pods (starting from kube 1.12)

```
apiVersion: apps/v1
kind: DaemonSet
metadata:
  labels:
    app: elasticsearch
  name: elasticsearch
spec:
  selector:
    matchLabels:
      app: elasticsearch
  template:
    metadata:
      labels:
        app: elasticsearch
    spec:
      containers:
      - image: k8s.gcr.io/fluentd-elasticsearch:1.20
        name: fluentd-elasticsearch
        resources: {}
```        

## Static Pods

Pods created by kubelet by its own without scheduler or api-server

Pod definiton can be provided in a directory and kublet with create/update/delete pods based on poddefiniiton added/updated/deleted in the directory

`--pod-manifest-path`

or give a `--config=kubeconfig.yaml` to kubectl and spcify path in the config file

```
staticPodPath: /etc/kubernetes/manifests
```

static pods can be listed using `docker ps` command, (not kubectl command)

kubelet can create static pods and pods specified by apiServer+scheduler at the same time
if cluster is up, kubernetes will be aware of static pods. (can view, not edit or delete)

staticpods can be used to deploy controlplane components

node name is attached to podname

to list all static pods

```
kubectl get pods --all-namespaces

#then look for podnames with nodenames appened in them
```

finding staticPodPath

```
ps -aux | grep kubelet
```

